<a href="https://colab.research.google.com/github/Avinashjha360/Speech-Emotion-Recognizer/blob/main/Speech_Emotion_Recognition_with_librosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
Root = "/content/drive/MyDrive/Audio"
os.chdir(Root)

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
ls

Actor_01/  Actor_04/  Actor_07/  Actor_10/  Actor_13/                Static/
Actor_02/  Actor_05/  Actor_08/  Actor_11/  Actor_14/                templates/
Actor_03/  Actor_06/  Actor_09/  Actor_12/  modelForPrediction1.sav


In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Audio/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

array([[-5.43256897e+02,  5.28675842e+01, -4.83773422e+00, ...,
         3.38275539e-04,  1.65842270e-04,  7.32426124e-05],
       [-5.95419983e+02,  5.12002335e+01,  1.01169562e+00, ...,
         4.36859002e-04,  2.21273469e-04,  1.56086942e-04],
       [-6.61117310e+02,  6.05738106e+01,  3.28022075e+00, ...,
         1.42650988e-05,  3.16007549e-06,  1.47057870e-06],
       ...,
       [-5.33280579e+02,  5.14346428e+01, -1.71162987e+01, ...,
         6.80597950e-05,  4.47286220e-05,  1.50650094e-05],
       [-6.41886963e+02,  3.98759880e+01, -7.34481382e+00, ...,
         1.42723831e-04,  1.07523367e-04,  5.06289107e-05],
       [-6.44309631e+02,  4.44270973e+01, -1.85680561e+01, ...,
         1.67694870e-05,  5.24664210e-06,  2.52183804e-06]])

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(336, 113)


In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['happy', 'calm', 'disgust', 'happy', 'calm', 'happy', 'happy',
       'disgust', 'calm', 'calm', 'fearful', 'fearful', 'fearful',
       'fearful', 'happy', 'disgust', 'disgust', 'calm', 'happy', 'happy',
       'fearful', 'fearful', 'happy', 'fearful', 'disgust', 'happy',
       'fearful', 'calm', 'happy', 'disgust', 'disgust', 'calm', 'calm',
       'happy', 'disgust', 'disgust', 'disgust', 'calm', 'fearful',
       'disgust', 'fearful', 'happy', 'fearful', 'happy', 'happy',
       'disgust', 'fearful', 'happy', 'happy', 'disgust', 'calm', 'calm',
       'calm', 'calm', 'calm', 'disgust', 'calm', 'fearful', 'happy',
       'happy', 'disgust', 'fearful', 'disgust', 'calm', 'happy', 'happy',
       'fearful', 'disgust', 'fearful', 'calm', 'calm', 'happy', 'happy',
       'disgust', 'happy', 'calm', 'happy', 'happy', 'calm', 'fearful',
       'calm', 'happy', 'calm', 'disgust', 'disgust', 'fearful',
       'fearful', 'happy', 'happy', 'calm', 'fearful', 'calm', 'disgust',
       

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.80%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.81355932, 0.6       , 0.64150943, 0.75      ])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,disgust,calm
2,calm,disgust
3,fearful,happy
4,calm,calm
5,happy,happy
6,fearful,happy
7,disgust,disgust
8,calm,calm
9,calm,calm


In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 4.9 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=41b2845dad7be2917342f9ee10a8f14fdb0c4f1f2af56dc33deecfe59e5b1fce
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
! ngrok authtoken xxxx

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [73]:
from flask_ngrok import run_with_ngrok
from flask import Flask , render_template, request
from markupsafe import escape
##Running the fask app
app = Flask(__name__)
##We need to start ngrok when the app is run
run_with_ngrok(app)

@app.route("/")
def index():
  return render_template('index.html', result = prediction[0])

@app.route('/', methods = ['POST'])
def update_text():
  filename = 'modelForPrediction1.sav'
  loaded_model = pickle.load(open(filename, 'rb'))
  # loading the model file from the storage
  url="/content/drive/MyDrive/Audio/Actor_01/03-01-08-02-02-01-01.wav"
  url2="/content/drive/MyDrive/Audio/Actor_01/"+request.form['audio']+".wav"
  feature=extract_feature(url2, mfcc=True, chroma=True, mel=True)
  feature=feature.reshape(1,-1)
  prediction=loaded_model.predict(feature)
  print(prediction[0])
  return render_template('index.html', result = prediction[0])
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://01fd-34-74-164-85.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [27/Nov/2022 20:17:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Nov/2022 20:18:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Nov/2022 20:18:04] "POST / HTTP/1.1" 200 -


happy


...

happy


INFO:werkzeug:127.0.0.1 - - [27/Nov/2022 20:21:19] "POST / HTTP/1.1" 200 -


calm


ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-73-f896e2cf70d0>", line 20, in update_text
    feature=extract_feature(url2, mfcc=True, chroma=True, mel=True)
  Fi

calm
